<a href="https://colab.research.google.com/github/AnaadKaur/Forest-Fire-Detection/blob/main/Forecasting%20Forest%20Fires.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Environment setup

## Run the two cells below for output wrap-around

In [2]:
from IPython.display import HTML, display

def set_css():
  display(HTML('''
  <style>
    pre {
        white-space: pre-wrap;
    }
  </style>
  '''))
get_ipython().events.register('pre_run_cell', set_css)

In [3]:
from notebook.services.config import ConfigManager
c = ConfigManager()
c.update('notebook', {"CodeCell": {"cm_config": {"lineWrapping": True}}})

{'CodeCell': {'cm_config': {'lineWrapping': True}}}

## Importing libraries

In [4]:
import math
import altair as alt
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
from pprint import pprint
import geemap
import matplotlib.pyplot as plt

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
import xgboost as xgb
import lightgbm as lgb
import statsmodels.api as sm
from scipy.stats import zscore

## Authentication and Initialization

In [5]:
import ee
ee.Authenticate()

In [6]:
ee.Initialize(project='ee-anaad')

## Loading Random Forest (Landsat-8)

In [7]:
def getFireMaskOldRandomForest(image):
    asset_id = 'projects/ee-anaad/assets/Classifiers/random_forest_classifier' # 75-25
    random_forest_classifier = ee.Classifier.load(asset_id)
    return image.classify(random_forest_classifier).rename(['F'])

# Time Series Analysis

In [8]:
district_boundaries = ee.FeatureCollection('projects/ee-anaad/assets/India_district_boundaries')
aez_boundaries = ee.FeatureCollection('projects/ee-anaad/assets/India_AEZ_boundaries')
block_boundaries = ee.FeatureCollection('projects/ee-anaad/assets/AEZ_mapped_block_boundaries')

In [ ]:
# Run the code below to visualize AEZ boundaries
Map = geemap.Map()
Map.addLayerControl()

Map.addLayer(aez_boundaries, {}, 'AEZs')
Map.addLayer(block_boundaries, {}, 'Blocks')

n = aez_boundaries.size().getInfo()
aez_list = aez_boundaries.toList(n)
for i in range(n):
    aez = ee.Feature(aez_list.get(i))
    Map.addLayer(aez, {}, 'aez'+str(aez.get('ae_regcode').getInfo()))

Map

## Helper Functions

In [15]:
bands_to_select = ['Tair_f_inst', 'Evap_tavg', 'Qair_f_inst', 'Rainf_f_tavg', 'SoilTMP0_10cm_inst', 'Wind_f_inst']
bands_add = ['Evap_tavg', 'Rainf_f_tavg']
bands_avg = ['Tair_f_inst', 'Qair_f_inst', 'SoilTMP0_10cm_inst', 'Wind_f_inst']

def get_data_row(aoi_geo, landsatImagePrefix, date_object, method):
    dict_row = {}

    getFireMask = getFireMaskOldRandomForest

    landsatImageId = landsatImagePrefix+date_object.strftime("%Y%m%d")
    try:
        ee.data.getAsset(landsatImageId)
    except ee.EEException:
        print(landsatImageId, 'does not exist.')
        return None
    image = ee.Image(landsatImageId).clip(aoi_geo)
    fire_mask = getFireMask(image)#.updateMask(forest_mask)
    fire = fire_mask.updateMask(fire_mask)
    fire_pixels = fire.reduceRegion(reducer = ee.Reducer.sum(), scale = 30, maxPixels = 1e8)
    num_fires = ee.Number(fire_pixels.get('F')).getInfo()
    # fire_vectors = fire.addBands(fire).reduceToVectors(
    #     geometry = aoi_buffered,
    #     scale = 30,
    #     reducer = ee.Reducer.mean(),
    #     maxPixels = 1e10
    # )
    # num_fires = fire_vectors.size().getInfo()
    dict_row['Num_fires'] = num_fires

    def get_climatic_data(ind):
        start_date = (date_object+timedelta(days=ind)).strftime("%Y-%m-%d")
        end_date = (date_object+timedelta(days=ind+1)).strftime("%Y-%m-%d")
        ic = ee.ImageCollection("NASA/GLDAS/V021/NOAH/G025/T3H")\
        .filterBounds(aoi_geo)\
        .filterDate(start_date, end_date)

        image1 = ic.select(bands_avg).mean().clip(aoi_geo)
        mean_values = image1.reduceRegion(reducer=ee.Reducer.mean(), geometry=aoi_geo)
        values = mean_values.getInfo()
        for key in values:
            dict_row[key+str(ind)] = values[key]

        image2 = ic.select(bands_add).sum().clip(aoi_geo)
        mean_values = image2.reduceRegion(reducer=ee.Reducer.mean(), geometry=aoi_geo)
        values = mean_values.getInfo()
        for key in values:
            dict_row[key+str(ind)] = values[key]*86400

    for ind in range(3):
        get_climatic_data(ind)

    return dict_row


def get_data_matrix(aoi, start_date, end_date, method):
    dict_rows = []
    aoi_geo = aoi.geometry()

    landsatImageCollection = ee.ImageCollection("LANDSAT/LC08/C02/T1_TOA")\
        .filterBounds(aoi_geo)\
        .filterDate(start_date, (datetime.strptime(start_date, "%Y-%m-%d")+timedelta(days=16)).strftime("%Y-%m-%d"))
    while(landsatImageCollection.size().getInfo() == 0):
        start_date = (datetime.strptime(start_date, "%Y-%m-%d")+timedelta(days=16)).strftime("%Y-%m-%d")
        landsatImageCollection = ee.ImageCollection("LANDSAT/LC08/C02/T1_TOA")\
        .filterBounds(aoi_geo)\
        .filterDate(start_date, (datetime.strptime(start_date, "%Y-%m-%d")+timedelta(days=16)).strftime("%Y-%m-%d"))

    n = landsatImageCollection.size().getInfo()
    landsatImageList = ee.List(landsatImageCollection.toList(n))
    max_intersection_area = 0
    landsatImageId = None
    for i in range(n):
        landsatImage = ee.Image(landsatImageList.get(i))
        intersection_area = landsatImage.geometry().intersection(aoi_geo).area().getInfo()
        if intersection_area > max_intersection_area:
            max_intersection_area = intersection_area
            landsatImageId = landsatImage.id().getInfo()
    date_string = landsatImageId[-8:]
    date_object = datetime.strptime(date_string, "%Y%m%d")
    end_date = datetime.strptime(end_date, "%Y-%m-%d")
    landsatImagePrefix = 'LANDSAT/LC08/C02/T1_TOA/'+landsatImageId[:11]+'_'

    adi = aoi.get('ADI 2019').getInfo()
    while date_object < end_date:
        dict_row = get_data_row(aoi_geo, landsatImagePrefix, date_object, method)
        if dict_row != None:
            dict_row['ADI'] = adi
            dict_rows.append(dict_row)
        date_object = date_object+timedelta(days=16)

    return pd.DataFrame(dict_rows)

# Use like this
# block = block_boundaries.filter(ee.Filter.eq('Subdistric', 'Masalia')).first()
# block_data = get_data_matrix(block, '2019-07-01', '2019-07-31', 'OldRandomForest')

## Testing

In [ ]:
blocks = block_boundaries.filter(ee.Filter.eq('aez_id', 12))

sampled_blocks = blocks.randomColumn()
sampled_blocks = sampled_blocks.filter('random < 0.03')
display(sampled_blocks)

In [ ]:
start_date = '2019-07-01'
end_date = '2022-07-01'
Map = geemap.Map()
Map.addLayerControl()
Map

In [ ]:
data = pd.DataFrame()

b = sampled_blocks.size().getInfo()
print(b)
blocks_list = sampled_blocks.toList(b)
for i in range(b):
    block = ee.Feature(blocks_list.get(i))
    print(block.get('Subdistric').getInfo())
    Map.addLayer(block, {}, 'block'+str(i))
    print('Generating data for block ....')
    block_data = get_data_matrix(block, '2019-07-01', '2022-07-01', 'OldRandomForest')
    print('done')
    data = pd.concat([data, block_data], ignore_index=True)

In [ ]:
positive_data = data[data['Num_fires'] > 0]
zero_data = data[data['Num_fires'] == 0]
zero_data = zero_data.sample(n = len(positive_data)).reset_index(drop=True)
balanced_data = pd.concat([positive_data, zero_data], ignore_index=True, sort=False)
balanced_data_z = balanced_data.apply(zscore)